# UB DataScience: Complex networks
### David Solans & Gerard Martinez

In [9]:

import networkx as nx
import matplotlib.pyplot as plt
from multiprocessing import Pool
import itertools
from networkx.algorithms import community
import collections
import numpy as np
import pickle
import operator
import folium
from scipy.optimize import curve_fit

plt.style.use('seaborn')

%matplotlib inline

## Loading data

In [10]:
G_transport = nx.read_edgelist("barcelona_transport.edgelist", create_using = nx.Graph(), nodetype = int)
G_bus = nx.read_edgelist("barcelona_bus.edgelist", create_using = nx.Graph(), nodetype = int)
G_metro = nx.read_edgelist("barcelona_metro.edgelist", create_using = nx.Graph(), nodetype = int)

In [11]:
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [12]:
bus_stations_properties = load_obj("bus_stations_properties")

## Week 6 - 

### Random walk

In [13]:
restart_prob=0.5 #To be checked
M = nx.adjacency_matrix(G_bus) #obtain the adj. matrix for the graph
#normalise the adjacency matrix
for i in range(M.shape[1]):
    if (np.sum(M[i]) > 0):
        M[i] = M[i]/np.sum(M[i])
p = 0#generate a random number between 0 and 1
if p < restart_prob:
    #do restart
    pass
else:
    #choose next node
    pass

### Kuramoto

### Dynamic process